This script takes two languages (usually english and another) and returns a list of pairs of sections names

In this example, we use as seed the high quality articles in english

In [2]:
import pandas as pd

### Reading wikidata information for article alingment 

In [3]:
il = pd.read_csv('interlanguageResult.txt.gz',sep='\t',header=None)

In [4]:
il = il.rename(columns={0:'wikidata',1:'lang',2:'page_title'})


In [5]:
selectedLanguages = il[il['lang'].isin(['pt','es','en','it','de'])]

selectedLanguages.set_index('wikidata')

In [7]:
selectedLanguages.columns

Index(['wikidata', 'lang', 'page_title'], dtype='object')

### Reading the ids of the high quality articles in English

(Class FL, FA, and A sccording https://en.wikipedia.org/wiki/Wikipedia:WikiProject_assessment)

In [8]:
articlesHq = pd.read_csv('articles-en-class-FL-FA-A.txt',sep='\t')

In [9]:
articlesHq = articlesHq.drop('cl_to',1)

In [10]:
articlesHq = articlesHq.drop_duplicates()

In [11]:
articlesHq['page_title'] = articlesHq.page_title.str.replace('_',' ')

In [12]:
selectedEn = pd.merge(selectedLanguages[selectedLanguages['lang']=='en'],articlesHq,on='page_title',how='inner')

In [13]:
selectedEn.shape

(34614, 4)

In [14]:
selectedEn.set_index('wikidata')

,lang,page_title,page_id
wikidata,,,
Q17,en,Japan,15573
Q21,en,England,9316
Q22,en,Scotland,26994
Q25,en,Wales,69894
Q30,en,United States,3434750
Q1,en,Universe,31880
Q2,en,Earth,9228
Q23,en,George Washington,11968
Q1971,en,Imagine (John Lennon song),18960141


In [15]:
selectedEs =  selectedLanguages[selectedLanguages['lang'] =='es'][selectedLanguages['wikidata'].isin(selectedEn.wikidata.tolist())]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [16]:
selectedPt =  selectedLanguages[selectedLanguages['lang'] =='pt'][selectedLanguages['wikidata'].isin(selectedEn.wikidata.tolist())]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [17]:
selectedIt =  selectedLanguages[selectedLanguages['lang'] =='it'][selectedLanguages['wikidata'].isin(selectedEn.wikidata.tolist())]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [18]:
selectedDe =  selectedLanguages[selectedLanguages['lang'] =='de'][selectedLanguages['wikidata'].isin(selectedEn.wikidata.tolist())]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [19]:
selectedEs.shape

(15248, 3)

In [20]:
selectedIt.shape

(13537, 3)

In [21]:
selectedPt.shape

(12842, 3)

In [52]:
import json, requests
idsEs = {}
titleEs = selectedEs.page_title.tolist()
redirectsEs = []
for i in range(0, len(titleEs), 50): #Api allows queries of 50 elements
    titles = '|'.join(titleEs[i:i+50])
    results = json.loads(requests.get('https://es.wikipedia.org/w/api.php?action=query&format=json&titles=%s&redirects=1' % titles).content.decode())
    for pageid, data in results['query']['pages'].items():
        idsEs[data['title']] = pageid
    try:
        redirectsPt.append(results['query']['redirects'])
        for r in results['query']['redirects']:
            idsEs[r['from']] = idsEs[r['to']]
    except: pass

In [20]:
import json, requests
idsPt = {}
titlePt = selectedPt.page_title.tolist()
redirectsPt = []
for i in range(0, len(titlePt), 50): #Api allows queries of 50 elements
    titles = '|'.join(titlePt[i:i+50])
    results = json.loads(requests.get('https://pt.wikipedia.org/w/api.php?action=query&format=json&titles=%s&redirects=1' % titles).content.decode())
    for pageid, data in results['query']['pages'].items():
        idsPt[data['title']] = pageid
    try: 
        redirectsPt.append(results['query']['redirects'])
        for r in results['query']['redirects']:
            idsPt[r['from']] = idsPt[r['to']]
    except: pass

In [21]:
import json, requests
idsIt = {}
titleIt = selectedIt.page_title.tolist()
redirectsIt = []
for i in range(0, len(titleIt), 50): #Api allows queries of 50 elements
    titles = '|'.join(titleIt[i:i+50])
    results = json.loads(requests.get('https://it.wikipedia.org/w/api.php?action=query&format=json&titles=%s&redirects=1' % titles).content.decode())
    for pageid, data in results['query']['pages'].items():
        idsIt[data['title']] = pageid
    try: 
        redirectsIt.append(results['query']['redirects'])
        for r in results['query']['redirects']:
            idsIt[r['from']] = idsIt[r['to']]
    except: pass

In [22]:
import json, requests
idsDe = {}
titleDe = selectedDe.page_title.tolist()
redirectsDe = []
for i in range(0, len(titleDe), 50): #Api allows queries of 50 elements
    titles = '|'.join(titleDe[i:i+50])
    results = json.loads(requests.get('https://de.wikipedia.org/w/api.php?action=query&format=json&titles=%s&redirects=1' % titles).content.decode())
    for pageid, data in results['query']['pages'].items():
        idsDe[data['title']] = pageid
    try: 
        redirectsDe.append(results['query']['redirects'])
        for r in results['query']['redirects']:
            idsDe[r['from']] = idsIt[r['to']]
    except: pass

In [58]:
idsPt['Polonio']

'5828'

In [22]:
import json
#commented to dont overwrite
#with open('ids-mapped-hq-pt.json','w') as f:
#    json.dump(idsPt,f)
#with open('ids-mapped-hq-es.json','w') as f:
#    json.dump(idsEs,f)
#with open('ids-mapped-hq-de.json','w') as f:
#    json.dump(idsDe,f)
#with open('ids-mapped-hq-it.json','w') as f:
#    json.dump(idsIt,f)

#uncomment to load
with open('ids-mapped-hq-es.json') as f: idsEs = json.load(f)
with open('ids-mapped-hq-pt.json') as f: idsPt = json.load(f)
with open('ids-mapped-hq-de.json') as f: idsDe = json.load(f)
with open('ids-mapped-hq-it.json') as f: idsIt = json.load(f)

# Find the most frequent sections

In [3]:
from collections import Counter
import gzip,json
sectionsEnAll = []
with gzip.open('sections-articles_en_all.json.gz') as f: #this for the freq full data
#with open('sections-hq-articles.json') as f: #this for the freq hq articles
    for l in f:
        tmp = json.loads(l.decode())
        sectionsEnAll.append(tmp[2])
sectionsEnFreq = Counter([item.strip() for sublist in sectionsEnAll for item in sublist])
sectionsEnFreq.most_common(50)

[('References', 4338110),
 ('External links', 2424864),
 ('See also', 1193058),
 ('History', 562076),
 ('Notes', 268312),
 ('Career', 189251),
 ('Biography', 159437),
 ('Further reading', 157293),
 ('Track listing', 148890),
 ('Cast', 115780),
 ('Early life', 112224),
 ('Geography', 110448),
 ('Personal life', 110118),
 ('Bibliography', 109084),
 ('Sources', 104154),
 ('Plot', 99574),
 ('Background', 95651),
 ('Reception', 91647),
 ('Description', 87241),
 ('Discography', 81772),
 ('Demographics', 80988),
 ('Personnel', 79837),
 ('Awards', 73351),
 ('Education', 62429),
 ('Filmography', 59301),
 ('Results', 57250),
 ('Life', 55997),
 ('Honours', 55419),
 ('Works', 49672),
 ('Gallery', 47838),
 ('Production', 47223),
 ('Overview', 37847),
 ('Charts', 36788),
 ('Death', 34850),
 ('Distribution', 34320),
 ('Footnotes', 32963),
 ('Club career', 30726),
 ('Publications', 30666),
 ('Early life and education', 30481),
 ('Legacy', 30231),
 ('Family', 29840),
 ('Population', 28897),
 ('Species'

In [24]:
total = sum(sectionsEnFreq.values())
acc =0
secsToEval = []
for n,(sec,freq) in enumerate(sectionsEnFreq.most_common()):
    acc+= freq
    secsToEval.append(sec)
    #print(n,acc/total,sec)

    if acc/total > 0.80:
        print(n)
        break

1252


In [25]:
sectionsEsAll = []
with open('sections-articles_es.json') as f: #this for the freq full data
#with open('sections-hq-articles.json') as f: #this for the freq hq articles
    for l in f:
        tmp = json.loads(l)
        sectionsEsAll.append(tmp[1])
sectionsEsFreq = Counter([item.strip() for sublist in sectionsEsAll for item in sublist])
sectionsEsFreq.most_common(150)

[('Referencias', 871532),
 ('Enlaces externos', 718516),
 ('Véase también', 279711),
 ('Historia', 138342),
 ('Bibliografía', 135297),
 ('Demografía', 113912),
 ('Biografía', 99358),
 ('Geografía', 83198),
 ('Distribución geográfica', 54089),
 ('Descripción', 40952),
 ('Notas', 38270),
 ('Palmarés', 33983),
 ('Clubes', 30156),
 ('Taxonomía', 28652),
 ('Trayectoria', 25800),
 ('Discografía', 23964),
 ('Filmografía', 21678),
 ('Argumento', 20349),
 ('Lista de canciones', 20286),
 ('Carrera', 20178),
 ('Características', 18210),
 ('Morfología', 17542),
 ('Obras', 16374),
 ('Hábitat', 16359),
 ('Notas y referencias', 14306),
 ('Reparto', 14043),
 ('Premios', 13909),
 ('Sinopsis', 13385),
 ('Especies', 12815),
 ('Economía', 12022),
 ('Obra', 11398),
 ('Palmarés internacional', 11343),
 ('Selección nacional', 10665),
 ('Fuentes', 10481),
 ('Resultados', 10356),
 ('Distribución y hábitat', 10184),
 ('Clasificación', 10159),
 ('Antecedentes', 9317),
 ('Personajes', 8660),
 ('Recepción', 8366),

In [26]:
sectionsEsFreq['Infraestructura']

1141

## Baseline Experiment

In [ ]:
import json
def readSectionFile(f):
    outputDic = {}
    with open(f) as tmp:
        for l in tmp:
            data  = json.loads(l)
            outputDic[str(data[0])] = [x.strip() for x in data[1]]
    return outputDic
sectionsEn = readSectionFile('sections-hq-articles.json')
sectionsEs = readSectionFile('sections-articles_es.json')
sectionsPt = readSectionFile('sections-articles_pt.json')
sectionsDe = readSectionFile('sections-articles_de.json')
sectionsIt = readSectionFile('sections-articles_it.json')



In [47]:
from functools import reduce
dfs = [selectedEs,selectedPt,selectedEn]
allSelected = reduce(lambda left,right: pd.merge(left,right,on='wikidata',how='outer'), dfs)
allSelected = allSelected.rename(columns={'page_title_y':'page_title_pt','lang_y':'lang_pt','page_title_x':'page_title_es','lang_x':'lang_es'})

In [157]:
import itertools
import networkx as nx
G = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(sectionsEn[data['page_id']],sectionsEs[idsEs[data['page_title']]]):
            try:
                G[x][y]['w']+=1
            except:
                G.add_edge(x,y)
                G[x][y]['w'] =1
    except: 
        pass

In [49]:
for a, b, data in sorted(G.edges(data=True), key=lambda x: x[2]['w'], reverse=True)[0:10]:
    print('{a} {b} {w}'.format(a=a, b=b, w=data['w']))

References Referencias 3840
References Enlaces externos 3685
External links Enlaces externos 3616
Referencias External links 3548
Enlaces externos See also 1448
Referencias See also 1446
References Véase también 1310
Véase también External links 1240
Referencias Notes 1082
Notes Enlaces externos 1067


In [154]:
for node in G.nodes():
        print(node,' ===> ',sorted(G[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0])

  ===>  Release history
Ruido  ===>  Loudspeaker equalisation
Marriage and suicide  ===>  Referencias
"No puedo dormir, me come el payaso"  ===>  Reception
St. Louis Cardinals and Browns  ===>  Referencias
Carreras post-Clash  ===>  History
Y Rhiw  ===>  History
Influencias en compositores posteriores  ===>  List of compositions
Early military service  ===>  Referencias
Wealthy collector  ===>  Referencias
Infancia y estudios en Polonia  ===>  Life
Presidencia de Bolivia  ===>  See also
Palestina  ===>  Early years
Simbología  ===>  External links
Feature films  ===>  Referencias
Image  ===>  Enlaces externos
Other  ===>  Bibliografía utilizada
Collaborations  ===>  Enlaces externos
Camino a la presidencia  ===>  Entertainment career
Baloncesto masculino  ===>  Athletic directors
Televisión  ===>  External links
Mecánica de juego  ===>  Reception
Seguimiento y creencias de manifestación  ===>  Mahatma Gandhi on Swaminarayan
Tres poemas de Emily Dickinson  ===>  Life
Acontecimientos  ==

Early life and travels  ===>  Obras destacadas
Post-senate career  ===>  Referencias
Places of worship  ===>  Religión
Viaje a México  ===>  Paleoecology
Perspectivas nazis  ===>  Usage
Microsoft Surface RT  ===>  History
Amenazas y protección  ===>  See also
Other formats  ===>  Lista de temas
Datos del club  ===>  External links
Política y Administración Pública  ===>  History
Influencias y estilo  ===>  External links
Alimentación  ===>  References
Confirmación como obispo  ===>  Education and marriage
Cultural and political image  ===>  Enlaces externos
Performance timelines  ===>  Torneos de Grand Slam
Militancia en el PDPA  ===>  Notes
Live performances and media usage  ===>  Antecedentes y anuncio
Recording and composition  ===>  Referencias
Escándalo  ===>  References
2016  ===>  References
Pre-World War I  ===>  Referencias
''Persona non grata''  ===>  Estudios
March  ===>  Legado
Mecánica del juego  ===>  Reception
Segunda guerra mundial  ===>  Interbellum and Korean War
Tech

Directing techniques  ===>  Carrera
Años 1970: de The Mothers a Zappa  ===>  Beliefs and politics
Momentos destacados  ===>  Roles
Member for Flinders, 1918–23  ===>  Referencias
Farsari and ''Yokohama shashin''  ===>  Referencias
El sistema planetario  ===>  See also
Discography Faada Freddy  ===>  Discografía
Epónimo  ===>  In other media
Performance history  ===>  Referencias
Increíble: El Yankee Stadium se rindió ante Derek Jeter  ===>  Career highlights
Style of play and legacy  ===>  Trayectoria deportiva
Family and interests  ===>  Enlaces externos
Rosie Ribbons version  ===>  Referencias
Taxonomy, naming, and classification  ===>  Enlaces externos
Software and operating systems  ===>  Referencias
Primera fase  ===>  Notes
Actuaciones en vivo y versiones de otros artistas  ===>  Composition and lyrics
Reparto y doblaje en España  ===>  Release
Prólogo  ===>  Reception
Molestias y racismo  ===>  References
Afluencias de público  ===>  History
Problemas financieros  ===>  Early ye

Tour and John Frusciante's departure  ===>  Referencias
Industria  ===>  History
Birth of Panaflex  ===>  Referencias
Fanbase and marketing  ===>  Referencias
Works by Disraeli  ===>  Referencias
Copyright infringement suit  ===>  Trasfondo
Bob Dylan's versions  ===>  Referencias
Theological and political positions  ===>  Referencias
Specifications<!-- (Rafale which variant?) -->  ===>  Especificaciones
Musical styles and legacy  ===>  Carrera como actriz
Setting and plot  ===>  Referencias
Title and packaging  ===>  Referencias
Captivity  ===>  Referencias
Promoción y presentaciones en vivo  ===>  Release history
Film style  ===>  Comienzos en el teatro
Imaginación popular  ===>  History
Departmental commander and Reconstruction  ===>  Sherman y la Guerra Total
Edificios  ===>  History
Solar energy project  ===>  Referencias
Torture and murder of slaves and 1834 LaLaurie mansion fire  ===>  Referencias
Size  ===>  Referencias
Sequels  ===>  Enlaces externos
Set de rodaje  ===>  Recept

Bioquímica  ===>  External links
Sources cited  ===>  Carrera
Patología  ===>  Pathology
Promoción y recepción  ===>  Release
Filmografía selecta  ===>  Early years
Prisoner of war  ===>  Enlaces externos
Producción y grabación  ===>  External links
Republican politics 1854–60  ===>  Referencias
2003-2008: Una lesión importante  ===>  Career highlights
''Resident Evil 6''  ===>  References
Professional wrestling careers  ===>  Carrera
Papel en la creación de [[Nupedia]] y [[Wikipedia]]  ===>  History
Listas de Canciones  ===>  Release history
Music and compositions  ===>  Referencias
Attacks on Puerto Principe and Porto Bello (1667–1668)  ===>  Saqueo de Portobelo
Lenguaje como instinto  ===>  Public debate
Colt's Patent Manufacturing Company (1847–1860)  ===>  Biografía
Reports of government harassment prior to 2011  ===>  Referencias
Ecology, habitat and distribution  ===>  Enlaces externos
U.S. Senate career  ===>  Posiciones políticas
Sistema de juego tradicional  ===>  Games
Avail

...

1985 hasta 1988  ===>  Notes
Lajoie en Cleveland  ===>  See also
Coleccionistas de libros  ===>  History
Diseño gráfico  ===>  References
Commands  ===>  Carrera militar
Training and Event Centre  ===>  Centros Scouts Internacionales
Description and disposition  ===>  Referencias
''Travels of Marco Polo''  ===>  Referencias
Final de la [[Anexo:Recopa de Europa 1998-99|Recopa de Europa 1998-99]]  ===>  History
Antes de la Primera Guerra Mundial  ===>  Notes
Woman’s rights orator  ===>  Referencias
Architect  ===>  Referencias
Contributions to chess  ===>  Referencias
Movies That Rides Opened For  ===>  Incidente
Kepler-11 y sistema planetario  ===>  Characteristics
Tours and residency shows  ===>  Legado
Establishment and early years  ===>  Formula 3000
Teología  ===>  Life
Franchise records  ===>  Referencias
Tomb Raider 3 Gold: El Artefacto Perdido, Tomb Raider 3 Gold: The Lost Artifact  ===>  Reception
Evaluaciones sobre el personaje  ===>  Historical sources
Mariculture  ===>  Refer

World War I and interwar years  ===>  Referencias
Especies amenazadas  ===>  Breeding and propagation
Opening  ===>  Referencias
NBA  ===>  External links
Awards in his name  ===>  Legado
Summit  ===>  Referencias
Re-recordings and cover versions  ===>  Lista de canciones
Luchas de Apuestas  ===>  In wrestling
Lenguaje de manipulación de datos  ===>  History
Chart performance and influence  ===>  Referencias
Comparación con ''Heat''  ===>  Remake
Orders and deliveries  ===>  Referencias
''The Essential''  ===>  Promoción
Biology and ecology  ===>  Referencias
Matrimonio y familia  ===>  Marriage and family
Voting history  ===>  Referencias
Visitas  ===>  Visiting
Early years and college career  ===>  Referencias
Beliefs and thought  ===>  Referencias
The Lost Weekend  ===>  Early years
History and naming  ===>  Referencias
Figura mediática y negocios  ===>  Early years
Desempeño en las listas de venta  ===>  Track listings and formats
''Criss Angel Believe''  ===>  Referencias
Contenid

John's later reign and service under King Henry III  ===>  Citas
The arts  ===>  Referencias
''Live action''  ===>  Story
Collaboration human-robot  ===>  Referencias
Identity and origins  ===>  Enlaces externos
Usos e impacto social  ===>  History
Stage design and show production  ===>  Emisiones y grabaciones
Economía e industria  ===>  History
Works  ===>  Enlaces externos
Early life and junior career  ===>  Referencias
Bell's Isotope-Modified Optical Fiber patent application  ===>  Referencias
Argumento del modo campaña  ===>  Reception
Communications  ===>  Referencias
Últimos conciertos  ===>  Sources
Impacto  ===>  References
Fundación de Topeka y servicio militar  ===>  Founding Topeka and military service
Cover versions and samples  ===>  Créditos
Legado e impacto  ===>  Reception
Obras citadas  ===>  See also
Neighbourhoods  ===>  Referencias
Co-premiership (1993–1997)  ===>  Referencias
El Show  ===>  Critical response
Jefe de Gobierno del Distrito Federal  ===>  Notes
La in

Imagen  ===>  External links
Jaguar XJ220 Pininfarina  ===>  Enlaces externos
Incidente durante los MTV Video Music Awards del 2000  ===>  History
Performance style  ===>  Notas y referencias
Costumbres  ===>  History
Nuevos puestos  ===>  Bibliography
Compositions  ===>  Referencias
Highwayman  ===>  Referencias
Formation, 1954–1957  ===>  Referencias
Carrera como luchadora  ===>  In wrestling
Taxonomy and botanical history  ===>  Cultivo
Company history  ===>  Referencias
Posición en el horizonte de Chicago  ===>  In popular culture
Early years and early education  ===>  Bibliografía
Gubernatorial election of 1820  ===>  Referencias
Obra en español  ===>  Notes
Presentación en directo y versiones  ===>  Track listing and formats
2013–2015  ===>  Referencias
General  ===>  2nd generation
Common elements  ===>  Referencias
Notable natives  ===>  Referencias
Death of Michael Jackson  ===>  Michael Jackson´s Gone Too Soon
Childhood and early career  ===>  Referencias
Discursos  ===>  Acc

Otros artículos  ===>  Charlie Hebdo shootings
Aclaraciones  ===>  Notes
Themes, comparisons and influences on other franchises  ===>  Fuentes
USA for Africa musicians  ===>  Referencias
"Lost Weekend"  ===>  Referencias
Observational evidence  ===>  Evidencias
Pérdida de poder y muerte  ===>  Early years
Critical reception and impact  ===>  Acontecimientos
Posible predicción  ===>  Reception
Player records  ===>  Sistema de campeonato
Major collections  ===>  Referencias
Conception and creation  ===>  Referencias
Live performances and cover version  ===>  Referencias
''Daara J'' and ''Xalima''  ===>  Discografía
Familia Geller  ===>  Role
The concert  ===>  Lista de canciones
Country Outlaw  ===>  Recognition
DOBLE DQ  ===>  In wrestling
Premios y títulos  ===>  Early life and inspiration
Geología  ===>  External links
Queen of Palmyra  ===>  Referencias
Críticos de Marx  ===>  Life
Game summary  ===>  Referencias
Hijos  ===>  Historians' assessment
Lista de Pistas  ===>  References
R

In [52]:
for node in secsToEval:
    try:
        print(node,' ===> ',sorted(G[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0])
    except:
        pass

References  ===>  Referencias
External links  ===>  Enlaces externos
See also  ===>  Enlaces externos
Notes  ===>  Referencias
History  ===>  Enlaces externos
Reception  ===>  Referencias
Background  ===>  Referencias
Further reading  ===>  Enlaces externos
Plot  ===>  Enlaces externos
Production  ===>  Referencias
Bibliography  ===>  Enlaces externos
Early life  ===>  Referencias
Footnotes  ===>  Enlaces externos
Legacy  ===>  Enlaces externos
Description  ===>  Referencias
Aftermath  ===>  Enlaces externos
Charts  ===>  Referencias
Personal life  ===>  Enlaces externos
Track listing  ===>  Referencias
Critical reception  ===>  Referencias
Development  ===>  Enlaces externos
Sources  ===>  Referencias
Personnel  ===>  Referencias
Release  ===>  Referencias
Route description  ===>  Enlaces externos
Gameplay  ===>  Enlaces externos
Composition  ===>  Referencias
Credits and personnel  ===>  Referencias
Major intersections  ===>  Referencias
Cast  ===>  Enlaces externos
Release history  

## Removing node after recommend

In [58]:
import itertools
import networkx as nx
G_one = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(sectionsEn[data['page_id']],sectionsEs[idsEs[data['page_title']]]):
            try:
                G_one[x][y]['w']+=1
            except:
                G_one.add_edge(x,y)
                G_one[x][y]['w'] =1
    except: 
        pass

In [59]:
for node in secsToEval:
    try:
        rec = sorted(G_one[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
        print(node,' ===> ',rec)
        G_one.remove_node(rec)
    except:
        pass

References  ===>  Referencias
External links  ===>  Enlaces externos
See also  ===>  Véase también
Notes  ===>  Bibliografía
History  ===>  Historia
Reception  ===>  Recepción
Background  ===>  Lista de canciones
Further reading  ===>  Biografía
Plot  ===>  Argumento
Production  ===>  Producción
Bibliography  ===>  Notas
Early life  ===>  Filmografía
Footnotes  ===>  Historial de servicio
Legacy  ===>  Legado
Description  ===>  Descripción
Aftermath  ===>  Resultados
Charts  ===>  Antecedentes
Personal life  ===>  Vida personal
Track listing  ===>  Personal
Critical reception  ===>  Posicionamiento en listas
Development  ===>  Desarrollo
Sources  ===>  Discografía selecta
Personnel  ===>  Créditos
Release  ===>  Sinopsis
Route description  ===>  Cruces
Gameplay  ===>  Personajes
Composition  ===>  Composición
Credits and personnel  ===>  Video musical
Major intersections  ===>  Mantenimiento
Cast  ===>  Reparto
Release history  ===>  Historial de lanzamientos
Music video  ===>  Vídeo m

### Considering section position (distance)

In [282]:
import itertools
import networkx as nx
G_one_dist = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(enumerate(sectionsEn[data['page_id']]),
                                      enumerate(sectionsEs[idsEs[data['page_title']]])):
            try:
                G_one_dist[x[1]][y[1]]['w']+= 1-(abs(x[0]-y[0])/len(sectionsEn))
            except:
                G_one_dist.add_edge(x[1],y[1])
                G_one_dist[x[1]][y[1]]['w'] = 1-(abs(x[0]-y[0])/len(sectionsEn))
    except: 
        pass

In [283]:
for node in secsToEval:
    try:
        rec = sorted(G_one_dist[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
        print(node,' ===> ',rec)
        G_one_dist.remove_node(rec)
    except:
        pass

References  ===>  Referencias
External links  ===>  Enlaces externos
See also  ===>  Véase también
Notes  ===>  Bibliografía
History  ===>  Historia
Reception  ===>  Recepción
Background  ===>  Lista de canciones
Further reading  ===>  Biografía
Plot  ===>  Argumento
Production  ===>  Producción
Bibliography  ===>  Notas
Early life  ===>  Filmografía
Footnotes  ===>  Historial de servicio
Legacy  ===>  Legado
Description  ===>  Descripción
Aftermath  ===>  Resultados
Charts  ===>  Antecedentes
Personal life  ===>  Vida personal
Track listing  ===>  Personal
Critical reception  ===>  Posicionamiento en listas
Development  ===>  Desarrollo
Sources  ===>  Discografía selecta
Personnel  ===>  Créditos
Release  ===>  Sinopsis
Route description  ===>  Pare leer
Gameplay  ===>  Personajes
Composition  ===>  Composición
Credits and personnel  ===>  Video musical
Major intersections  ===>  Cruces
Cast  ===>  Reparto
Release history  ===>  Historial de lanzamientos
Music video  ===>  Vídeo music

In [239]:
### Comparision one (deleting after recomending)

In [70]:
import itertools
import networkx as nx
G_one = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(sectionsEn[data['page_id']],sectionsEs[idsEs[data['page_title']]]):
            try:
                G_one[x][y]['w']+=1
            except:
                G_one.add_edge(x,y)
                G_one[x][y]['w'] =1
    except: 
        pass

G_one_dist = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(enumerate(sectionsEn[data['page_id']]),
                                      enumerate(sectionsEs[idsEs[data['page_title']]])):
            try:
                G_one_dist[x[1]][y[1]]['w']+= 1-(abs(x[0]-y[0])/max(len(sectionsEn),len(sectionsEs)))
            except:
                G_one_dist.add_edge(x[1],y[1])
                G_one_dist[x[1]][y[1]]['w'] = 1-(abs(x[0]-y[0])//max(len(sectionsEn),len(sectionsEs)))
    except: 
        pass

In [71]:
output = {'en':[],'euristic1':[],'euristic2':[]}
for node in secsToEval:
    try:
        rec = sorted(G_one[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
        rec_dist = sorted(G_one_dist[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
        print(node,' ===> ',rec,rec_dist)
        G_one.remove_node(rec)
        G_one_dist.remove_node(rec_dist)
        output['en'].append(node)
        output['euristic1'].append(rec)
        output['euristic2'].append(rec_dist)
    except:
        pass

References  ===>  Referencias Referencias
External links  ===>  Enlaces externos Enlaces externos
See also  ===>  Véase también Véase también
Notes  ===>  Bibliografía Bibliografía
History  ===>  Historia Historia
Reception  ===>  Recepción Recepción
Background  ===>  Lista de canciones Lista de canciones
Further reading  ===>  Biografía Biografía
Plot  ===>  Argumento Argumento
Production  ===>  Producción Producción
Bibliography  ===>  Notas Notas
Early life  ===>  Filmografía Filmografía
Footnotes  ===>  Historial de servicio Historial de servicio
Legacy  ===>  Legado Legado
Description  ===>  Descripción Descripción
Aftermath  ===>  Resultados Resultados
Charts  ===>  Antecedentes Antecedentes
Personal life  ===>  Vida personal Vida personal
Track listing  ===>  Personal Personal
Critical reception  ===>  Posicionamiento en listas Posicionamiento en listas
Development  ===>  Desarrollo Desarrollo
Sources  ===>  Discografía selecta Discografía selecta
Personnel  ===>  Créditos Crédi

Composition and lyrics  ===>  Lista de éxitos Lista de éxitos
Religion  ===>  Personajes ilustres Personajes ilustres
Events  ===>  Eventos Eventos
Significance  ===>  Atribución Atribución
Infrastructure  ===>  Enlaces Externos Ciudades hermanas
Assessment  ===>  Fuentes y bibliografía Fuentes y bibliografía
Causes  ===>  Pronóstico Etiología
Management  ===>  Epidemiología Prevención
Operation  ===>  Cierre Ficha
Status  ===>  Especies Estado de conservación
Musical style  ===>  Estilo musical Estilo musical
Band members  ===>  Miembros de la banda Miembros de la banda
Explanatory notes  ===>  Notas explicativas Notas explicativas
Transportation  ===>  Arte y cultura Arte y cultura
Operators  ===>  Operadores Operadores
Recording and composition  ===>  Interpretaciones en directo Información general
Criticism  ===>  Comunidad Perl Comunidad Perl
First World War  ===>  Final Final
International play  ===>  Habilidades Habilidades
Controversies  ===>  Controversia Controversia
Reign  =

In [253]:
df = pd.DataFrame(output)
df.to_excel('spanish.xls')

### Euristic 3, navegating the graph

In [510]:
import itertools
import networkx as nx
#returns the neighbord in position pos, sorted by weight [w]
def getNneighbor(Graph,node,pos):
    return sorted(Graph[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[pos][0]
    
G_nav = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(sectionsEn[data['page_id']],sectionsEs[idsEs[data['page_title']]]):
            try:
                G_nav[x][y]['w']+=1
            except:
                G_nav.add_edge(x,y)
                G_nav[x][y]['w'] =1
    except: 
        pass

for node in secsToEval:
    try:
        candidate = 0
        while candidate < 10 or candidate == len(G_nav[node]):
            rec = getNneighbor(G_nav,node,candidate)
            #print(node,rec,getNneighbor(G_nav,rec,candidate))
            if getNneighbor(G_nav,rec,0) == node:
                G_nav.remove_node(rec)
                G_nav.remove_node(node)
                break
            else:
                candidate +=1
                rec = 'Not found'
        print(node,' ===> ',rec)
    except:
        print(node,'Error')


References  ===>  Referencias
External links  ===>  Enlaces externos
See also  ===>  Véase también
History  ===>  Historia
Notes  ===>  Bibliografía
Career  ===>  Not found
Biography  ===>  Not found
Further reading  ===>  Notas
Track listing  ===>  Lista de canciones
Cast  ===>  Not found
Early life  ===>  Primeros años
Geography  ===>  Not found
Personal life  ===>  Biografía
Bibliography  ===>  Not found
Sources  ===>  Not found
Plot  ===>  Argumento
Background  ===>  Antecedentes
Reception  ===>  Recepción
Description  ===>  Descripción
Discography  ===>  Discografía
Demographics  ===>  Not found
Personnel  ===>  Personal
Awards  ===>  Condecoraciones
Education  ===>  Educación
Filmography  ===>  Filmografía
Results  ===>  Resultados
Life  ===>  Vida
Honours  ===>  Clubes
Works  ===>  Obras
Gallery  ===>  Galería
Production  ===>  Producción
Overview  ===>  MC12 Corsa
Charts  ===>  Posicionamiento en listas
Death  ===>  Muerte
Distribution  ===>  Not found
Footnotes  ===>  Not foun

In [148]:
G

['References',
 'External links',
 'See also',
 'Notes',
 'History',
 'Reception',
 'Background',
 'Further reading',
 'Plot',
 'Production',
 'Bibliography',
 'Early life',
 'Footnotes',
 'Legacy',
 'Description',
 'Aftermath',
 'Charts',
 'Personal life',
 'Track listing',
 'Critical reception',
 'Development',
 'Sources',
 'Personnel',
 'Release',
 'Route description',
 'Gameplay',
 'Composition',
 'Credits and personnel',
 'Major intersections',
 'Cast',
 'Meteorological history',
 'Release history',
 'Music video',
 'Design',
 'Career',
 'Taxonomy',
 'Notes and references',
 'Chart performance',
 'Live performances',
 'Cultural references',
 'Certifications',
 'Service history',
 'Key',
 'Release and reception',
 'Citations',
 'Geography',
 'Distribution and habitat',
 'Impact',
 'Commercial performance',
 'Biography',
 'Discography',
 'Themes',
 'Music',
 'Synopsis',
 'Career statistics',
 'Economy',
 'Filmography',
 'Later life',
 'Awards',
 'Battle',
 'Characteristics',
 'Educa

In [254]:
### Portuguese

In [208]:
import itertools
import networkx as nx
G_one = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(sectionsEn[data['page_id']],sectionsPt[idsPt[data['page_title']]]):
            try:
                G_one[x][y]['w']+=1
            except:
                G_one.add_edge(x,y)
                G_one[x][y]['w'] =1
    except: 
        pass

G_one_dist = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(enumerate(sectionsEn[data['page_id']]),
                                      enumerate(sectionsPt[idsPt[data['page_title']]])):
            try:
                G_one_dist[x[1]][y[1]]['w']+= 1-(abs(x[0]-y[0])/max(len(sectionsEn),len(sectionsPt)))
            except:
                G_one_dist.add_edge(x[1],y[1])
                G_one_dist[x[1]][y[1]]['w'] = 1-(abs(x[0]-y[0])//max(len(sectionsEn),len(sectionsPt)))
    except: 
        pass
    
output = {'en':[],'euristic1':[],'euristic2':[]}
for node in secsToEval:
    try:
        rec = sorted(G_one[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
        rec_dist = sorted(G_one_dist[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0] 
        print(node,' ===> ',rec,rec_dist)
        G_one.remove_node(rec)
        G_one_dist.remove_node(rec_dist)
        output['en'].append(node)
        output['euristic1'].append(rec)
        output['euristic2'].append(rec_dist)
    except:
        pass
df = pd.DataFrame(output)
df.to_excel('portuguese.xls')

References  ===>  Ligações externas Ligações externas
External links  ===>  {{Ligações externas}} {{Ligações externas}}
See also  ===>  Ver também Ver também
Notes  ===>  Bibliografia Bibliografia
History  ===>  História História
Reception  ===>  Recepção Recepção
Background  ===>  Faixas Faixas
Further reading  ===>  Biografia Biografia
Plot  ===>  Enredo Enredo
Production  ===>  Produção Produção
Bibliography  ===>  Legado Legado
Early life  ===>  Carreira Carreira
Footnotes  ===>  Discografia Discografia
Legacy  ===>  Jogabilidade Jogabilidade
Description  ===>  Descrição Descrição
Aftermath  ===>  Resultados Resultados
Charts  ===>  Créditos Créditos
Personal life  ===>  Filmografia Filmografia
Track listing  ===>  Desempenho nas tabelas musicais Desempenho nas tabelas musicais
Critical reception  ===>  Histórico de lançamento Histórico de lançamento
Development  ===>  Desenvolvimento Desenvolvimento
Sources  ===>  Referências Referências
Personnel  ===>  Antecedentes Antecedentes


Publication and reception  ===>  ''Before Watchmen'' ''Before Watchmen''
Taxonomy and evolution  ===>  Predadores Predadores
Home release  ===>  Resumo Resumo
Notable residents  ===>  Divisão administrativa Divisão administrativa
Guest appearances  ===>  Na luta profissional Na luta profissional
Theories  ===>  A família dos Vanir A família dos Vanir
Sister cities  ===>  Transportes Transportes
Conception  ===>  Datas da turnê Datas da turnê
Production and release  ===>  Receção pela crítica Receção pela crítica
Video games  ===>  Trilha sonora Trilha sonora
Philanthropy  ===>  Filantropia Filantropia
Premise  ===>  Filmes ''Spin-Offs''
Impact and legacy  ===>  Impacto e legado Impacto e legado
Today  ===>  Hoje Hoje
Other versions  ===>  Letra Letra
Popular culture  ===>  {{Referências externas}} Controvérsia
Character development  ===>  Trajetória Trívia
Family and early life  ===>  Família e início da vida Família e início da vida
Post-war  ===>  Pós Guerra Pós Guerra
Editions  ===>

# Studying the position of sections, and how to take adventaje of that

In [243]:
from collections import Counter
def fashion(inlist):
    return Counter(inlist).most_common(1)[0][0]

In [221]:
secPosEn = {}
for secs in sectionsEn.values():
    for pos,secName in enumerate(secs):
        secPosEn[secName] = secPosEn.get(secName,[])
        secPosEn[secName].append(pos)


In [246]:
import numpy as np
for sec in secsToEval[0:15]:
    print(sec,round(np.mean(secPosEn[sec]),1),np.median(secPosEn[sec]),fashion(secPosEn[sec]))

References 5.6 5.0 4
External links 7.0 6.0 5
See also 5.1 4.0 3
Notes 5.1 5.0 4
History 0.8 0.0 0
Reception 2.9 3.0 2
Background 0.1 0.0 0
Further reading 7.9 7.0 6
Plot 0.2 0.0 0
Production 1.3 1.0 1
Bibliography 6.8 6.0 5
Early life 0.0 0.0 0
Footnotes 4.5 4.0 3
Legacy 4.4 4.0 3
Description 0.7 1.0 1


In [230]:
secPosEs = {}
for secs in sectionsEs.values():
    for pos,secName in enumerate(secs):
        secPosEs[secName] = secPosEs.get(secName,[])
        secPosEs[secName].append(pos)


Notas 3.3 3.0


In [247]:
sec = 'Bibliografía'
print(sec,round(np.mean(secPosEs[sec]),1),np.median(secPosEs[sec]),fashion(secPosEs[sec]))
sec = 'Notas'
print(sec,round(np.mean(secPosEs[sec]),1),np.median(secPosEs[sec]),fashion(secPosEs[sec]))

Bibliografía 3.6 3.0 3
Notas 3.3 3.0 2


### Check typical lenght 
when section X appears, number of sections is usually N

In [235]:
secLenght = {}
for secs in sectionsEn.values():
    for secName in secs:
        secLenght[secName] = secLenght.get(secName,[])
        secLenght[secName].append(len(secs))

In [248]:
for sec in secsToEval[0:15]:
    print(sec,round(np.mean(secLenght[sec]),1),np.median(secLenght[sec]),fashion(secLenght[sec]))

References 7.5 7.0 6
External links 8.1 7.0 6
See also 8.4 8.0 6
Notes 8.1 7.0 7
History 7.7 7.0 6
Reception 7.2 7.0 6
Background 8.2 8.0 7
Further reading 9.7 9.0 8
Plot 6.9 6.0 6
Production 7.1 6.0 6
Bibliography 8.8 8.0 7
Early life 8.5 8.0 8
Footnotes 7.3 6.0 6
Legacy 9.4 9.0 8
Description 7.1 7.0 7


In [249]:
secLenghtEs = {}
for secs in sectionsEs.values():
    for secName in secs:
        secLenghtEs[secName] = secLenghtEs.get(secName,[])
        secLenghtEs[secName].append(len(secs))

In [250]:
sec = 'Bibliografía'
print(sec,round(np.mean(secLenghtEs[sec]),1),np.median(secLenghtEs[sec]),fashion(secLenghtEs[sec]))
sec = 'Notas'
print(sec,round(np.mean(secLenghtEs[sec]),1),np.median(secLenghtEs[sec]),fashion(secLenghtEs[sec]))

Bibliografía 5.4 5.0 5
Notas 5.6 5.0 5


### most usual predecesor 

In [255]:
secPredEn = {}
for secs in sectionsEn.values():
    for pos,secName in enumerate(secs[1:]):
        secPredEn[secName] = secPredEn.get(secName,[])
        secPredEn[secName].append(secs[pos-1])

In [257]:
for sec in secsToEval[0:15]:
    print(sec,fashion(secPredEn[sec]))

References See also
External links See also
See also History
Notes Aftermath
History External links
Reception Plot
Background External links
Further reading Notes
Plot External links
Production External links
Bibliography Notes
Early life External links
Footnotes See also
Legacy Release
Description External links


In [260]:
Counter(secPredEn['Notes']).most_common(5)

[('Aftermath', 284),
 ('Reception', 269),
 ('Design', 239),
 ('Legacy', 239),
 ('History', 196)]

In [ ]:
## Check when both langs have same number of sections

In [265]:
yes = 0
no = 0
for index,data in allSelected.iterrows():
    try:
        if len(sectionsEs[idsEs[data['page_title']]]) == len(sectionsEn[data['page_id']]):
            yes +=1
            print(sectionsEs[idsEs[data['page_title']]],sectionsEn[data['page_id']])
        else:
            no +=1
    except: pass


['Primeros años y vida privada', 'Carrera', 'Fallecimiento', 'Legado', 'Filmografía', 'Clasificación', 'Recepción y crítica', 'Premios', 'Referencias', 'Enlaces externos'] ['Early life', 'Photographic career', 'Film career', 'Career influences', 'Directing techniques', 'Personal life', 'Legacy', 'See also', 'References', 'External links']
['Política', 'Primeros años', 'Inicios en el nazismo', 'Ascenso al poder', 'Establecimiento de la dictadura', 'Tercer Reich', 'Segunda Guerra Mundial', 'Últimos días', 'Rasgos de su personalidad', 'Legado de Hitler', 'Véase también', 'Notas', 'Referencias', 'Bibliografía', 'Enlaces externos'] ['Early years', 'Entry into politics', 'Rise to power', 'Nazi Germany', 'World War II', 'Leadership style', 'Legacy', 'Views on religion', 'Health', 'Family', 'In propaganda films', 'See also', 'Notes', 'References', 'External links']
['Biografía', 'Never Ending Tour', 'Influencia, importancia y reconocimiento internacional', 'Pinturas y dibujos', 'Vida personal'

['Trayectoria', 'Selección nacional', 'Estadísticas', 'Palmarés', 'Condecoraciones', 'Estilo de juego', 'Vida privada', 'Referencias', 'Enlaces externos'] ['Club career', 'International career', 'Playing style', 'Personal life', 'Career statistics', 'Honours', 'Notes', 'References', 'External links']
['Primeros años', 'Carrera', 'Filmografía', 'Premios', 'Vida personal', 'Véase también', 'Referencias', 'Enlaces externos'] ['Early life', 'Career', 'Personal life', 'Filmography', 'Awards', 'Bibliography', 'References', 'External links']
['Primeros años', 'Establecimiento del nuevo orden', 'Apogeo del nuevo orden', 'Protestas por la reforma y caída de Suharto', 'Muerte', 'Notas', 'Referencias', 'Bibliografía', 'Enlaces externos'] ['Early life', 'Military career', 'Overthrow of Sukarno (1965)', 'The "New Order" (1967–1998)', 'Post-presidency', 'See also', 'References', 'Bibliography', 'External links']
['Biografía', 'Carrera como actriz', 'Carrera musical', 'Otros proyectos', 'Filmografía'

['Biografía', 'Muerte y leyenda', 'Reconocimientos', 'Lista de más famosas canciones', 'Véase también', 'Referencias', 'Enlaces externos'] ['Life and career', 'Legacy', 'Awards', 'Discography', 'References', 'Further reading', 'External links']
['Historia', 'Influencias y estilo', 'Nominaciones a los Premios Grammy', 'Miembros', 'Discografía', 'Véase también', 'Referencias', 'Enlaces externos'] ['History', 'Musical style and influences', 'Band members', 'Discography', 'Awards and nominations', 'See also', 'References', 'External links']
['Biografía', 'Obra', 'Publicaciones', 'Referencias', 'Enlaces externos'] ['Life and career', 'Writing', 'Reception and reputation', 'Notes, references and sources', 'External links']
['Árpád, el hijo de Álmos', 'Guerra contra el Imperio búlgaro', 'Descendencia', 'Véase también', 'Bibliografía', 'Referencias', 'Enlaces externos'] ['Biography', 'Legacy', 'Family', 'See also', 'Footnotes', 'References', 'External links']
['Trasfondo', 'Desarrollo', 'Graba

In [264]:
no

4801

In [ ]:
### using lenght information

In [417]:
##
import itertools
import networkx as nx
G_one_dist = nx.Graph()
for index,data in allSelected.iterrows():
    try:  #en to es
        for x, y in itertools.product(enumerate(sectionsEn[data['page_id']]),
                                      enumerate(sectionsEs[idsEs[data['page_title']]])):
            if len(sectionsEn[data['page_id']]) == len(sectionsEs[idsEs[data['page_title']]]):
                factor = 100
            else:
                factor = 1
            try:
                G_one_dist[x[1]][y[1]]['w']+= factor*1-(abs(x[0]-y[0])/len(sectionsEn))
            except:
                G_one_dist.add_edge(x[1],y[1])
                G_one_dist[x[1]][y[1]]['w'] = factor*1-(abs(x[0]-y[0])/len(sectionsEn))
    except: 
        pass

for node in secsToEval:
    try:
        rec = sorted(G_one_dist[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
        print(node,' ===> ',rec)
        G_one_dist.remove_node(rec)
    except:
        pass

References  ===>  Referencias
External links  ===>  Enlaces externos
See also  ===>  Véase también
History  ===>  Historia
Notes  ===>  Notas
Career  ===>  Carrera
Biography  ===>  Biografía
Further reading  ===>  Bibliografía
Track listing  ===>  Lista de canciones
Cast  ===>  Producción
Early life  ===>  Filmografía
Geography  ===>  Geografía
Personal life  ===>  Vida personal
Bibliography  ===>  Legado
Sources  ===>  Notas y referencias
Plot  ===>  Recepción
Background  ===>  Antecedentes
Reception  ===>  Referencias culturales
Description  ===>  Descripción
Discography  ===>  Discografía
Demographics  ===>  Demografía
Personnel  ===>  Personal
Awards  ===>  Premios
Education  ===>  Economía
Filmography  ===>  Primeros años
Results  ===>  Resultados
Life  ===>  Obra
Honours  ===>  Trayectoria
Works  ===>  Trabajos
Gallery  ===>  Galería
Production  ===>  Sinopsis
Overview  ===>  Resumen
Charts  ===>  Composición
Death  ===>  Muerte
Distribution  ===>  Taxonomía
Footnotes  ===>  Argu

Literary works  ===>  Su pensamiento sobre la salud y la alimentación
New Zealand  ===>  Nueva Zelanda
Importance  ===>  Importancia
2012  ===>  Personal life
International play  ===>  Post-Retiro
Basketball  ===>  Logros personales
Career achievements  ===>  Postura anti-dopaje y controversias
Literary career  ===>  Carrera literaria
Ideology  ===>  La masacre de estudiantes de Soweto (16 de junio de 1976)
Chronology  ===>  Cronología
Notable works  ===>  Trabajos notables
Pharmacology  ===>  Hábitat, distribuición, ecología
NBA career statistics  ===>  Galardones y logros
2011  ===>  Player profile
Parks and recreation  ===>  Servicios públicos
Users  ===>  Historial de combate
First World War  ===>  Reconocimientos por su edad
Restoration  ===>  Galería de imágenes
Videos  ===>  Career
Construction and career  ===>  Construcción y Carrera
Limitations  ===>  Diferencias
Sequels  ===>  Secuela
Samples  ===>  Posiciones en lista
2010  ===>  Professional career
Decline  ===>  Final
Arti

# Testing with ground truth

In [482]:
import pandas as pd
gt_es = pd.read_csv('spanish_gt.csv')[['en','es']]

In [483]:
gt_es.dropna().shape


(161, 2)

In [491]:
import itertools
import networkx as nx
for f in range(1,10,2):
    G_one_dist = nx.Graph()
    for index,data in allSelected.iterrows():
        try:  #en to es
            for x, y in itertools.product(enumerate(sectionsEn[data['page_id']]),
                                          enumerate(sectionsEs[idsEs[data['page_title']]])):
                if len(sectionsEn[data['page_id']]) == len(sectionsEs[idsEs[data['page_title']]]):
                    factor = f
                else:
                    factor = 1
                try:
                    G_one_dist[x[1]][y[1]]['w']+= factor*1-(abs(x[0]-y[0])/len(sectionsEn))
                except:
                    G_one_dist.add_edge(x[1],y[1])
                    G_one_dist[x[1]][y[1]]['w'] = factor*1-(abs(x[0]-y[0])/len(sectionsEn))
        except: 
            pass

    good = 0
    bad = 0
    for index,data in gt_es.dropna().iterrows():
            node = data['en']
            rec = sorted(G_one_dist[node].items(), key=lambda edge: edge[1]['w'],reverse=True)[0][0]
            G_one_dist.remove_node(rec)
            if data['es'] == rec:
                good +=1
            else:
                bad +=1

    print(f,good,bad)

1 85 76
3 81 80
5 80 81
7 80 81
9 80 81
